In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/"My Drive"/"Data Colab"/ml-pg-project

/content/drive/My Drive/Data Colab/ml-pg-project


In [ ]:
import pandas as pd
import torch.utils.data as Data
import torch.nn as nn
import numpy as np
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
TOTAL_FEATURES = 6
DAYS = 60
BATCH_SIZE = 128

In [ ]:
class StockMarketDataReg(Data.Dataset):
    def __init__(self, train=None, split=0.2, target_market="S&P", days=60):
        self.days = days + 1 # as last entry is to be predicted

        df = pd.read_csv(f"Processed_{target_market}.csv")
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        df = df.sort_values(by='Date')
        df = df.drop(columns=['Name', 'Date'])
        df = df.fillna(0)
        
        # outlier detection
        for col in df.columns:
            q1, q3 = df[col].quantile([0.25, 0.75])
            iqr = q3 - q1
            lb, rb = q1 - 1.5 * iqr, q3 + 1.5 * iqr
            for i in range(len(df)):
                if df[col][i] > rb:
                    df[col][i] = rb
                if df[col][i] < lb:
                    df[col][i] = lb

        self.feature_list = list(set(df.columns) - set(['Close']))
        if train is None:
            u = df.iloc[0: int(len(df) * (1 - split))]
            # pca
            self.pca = PCA(TOTAL_FEATURES - 1)
            self.pca_data = self.pca.fit_transform(u[self.feature_list])
            self.closing_prices = u['Close'].to_numpy()
            self.data = np.hstack(( self.closing_prices.reshape((-1, 1)), self.pca_data ))
        else:
            u = df.iloc[int(len(df) * (1 - split)) - self.days:]
            # pca
            self.pca_data = train.pca.transform(u[self.feature_list])
            self.closing_prices = u['Close'].to_numpy()
            self.data = np.hstack(( self.closing_prices.reshape((-1, 1)), self.pca_data ))

        self.num_rows = u.shape[0]
    
    def __getitem__(self, idx):
        end_idx = self.days + idx - 1
        market_tens = torch.from_numpy(self.data[idx: end_idx + 1])
        return market_tens.float()
    
    def __len__(self):
        return self.num_rows - self.days

In [ ]:
train_ds = StockMarketDataReg(target_market='S&P', split=0.2, days=DAYS)
test_ds = StockMarketDataReg(target_market='S&P', split=0.2, days=DAYS, train=train_ds)

In [ ]:
train_dataloader = Data.DataLoader(train_ds, batch_size=BATCH_SIZE)

In [ ]:
sample = next(iter(train_dataloader)).to(device)
sample.shape

torch.Size([128, 61, 6])

In [ ]:
class Generator(nn.Module):
    '''
    Generates fake data using features of past DAYS days
    '''
    def __init__(self, num_features, days, hidden_size=150):
        super().__init__()
        self.num_features = num_features
        self.days = days
        self.hidden_size = hidden_size
        # model
        self.lstm = nn.LSTM(input_size=self.num_features, hidden_size=self.hidden_size, num_layers=1, batch_first=True)
        self.decoder = nn.Sequential(
                nn.Linear(self.hidden_size * self.days, 2000),
                nn.BatchNorm1d(2000, momentum=0.9, eps=1e-05),
                nn.Tanh(),
                nn.Linear(2000, 500),
                nn.BatchNorm1d(500, momentum=0.9, eps=1e-05),
                nn.Tanh(),
                nn.Linear(500, 100),
                nn.ReLU(),
                nn.Linear(100, self.num_features),
                nn.ReLU()
            )
    def forward(self, inp):
        out1, _ = self.lstm(inp)
        out1 = out1.reshape((-1, self.hidden_size * self.days))
        out2 = self.decoder(out1)
        out2 = out2.squeeze()
        return out2

In [ ]:
class Discriminator(nn.Module):
    '''
    Discriminates between real data and fake data of DAYS + 1 days
    '''
    def __init__(self, num_features, c=5):
        # c denotes flatenning constant
        super().__init__()
        self.c = c
        self.num_features = num_features
        self.convs = nn.Sequential(
            nn.Conv1d(self.num_features, 32, kernel_size=5, stride=2),
            nn.LeakyReLU(0.01),
            nn.Conv1d(32, 64, kernel_size=5, stride=2),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(64, momentum=0.9, eps=1e-05), 
            nn.Conv1d(64, 128, kernel_size=5, stride=2),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(128, momentum=0.9, eps=1e-05),
        )
        self.dense = nn.Sequential(
            nn.Linear(128 * self.c, 220),
            nn.BatchNorm1d(220, momentum=0.9, eps=1e-05),
            nn.LeakyReLU(0.01),
            nn.Linear(220, 1),
            nn.Sigmoid()
        )
        self.linear = nn.Sequential(
            nn.Linear((DAYS + 1) * self.num_features, 200),
            nn.BatchNorm1d(200, momentum=0.9, eps=1e-05),
            nn.ReLU(), 
            nn.Linear(200, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )
    def forward(self, inp):
        inp = torch.transpose(inp, 2, 1)
        out1 = self.convs(inp)
        out1 = out1.reshape(-1, 128 * self.c)
        out2 = self.dense(out1)
        out2 = out2.squeeze()
        return out2
        # inp = inp.view(-1, (DAYS + 1) * self.num_features)
        # return self.linear(inp).squeeze()

In [ ]:
criterion = torch.nn.BCELoss()

generator = Generator(TOTAL_FEATURES, DAYS)
generator = generator.to(device)

discriminator = Discriminator(TOTAL_FEATURES)
discriminator = discriminator.to(device)
    
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.002)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.002)

In [ ]:
NUM_EPOCHS = 50

In [ ]:
for epoch in range(NUM_EPOCHS):
    running_G_loss, running_D_loss = 0, 0
    correct, cnt = 0, 0
    for data in tqdm(train_dataloader):
        #### discriminator 
        discriminator.zero_grad()
        data = data.to(device)
        real_prev, real_now = data[:, :-1], data[:, -1]
        labels = torch.ones(len(data)).float().to(device)
        
        ## real
        # forward
        output = discriminator(data)
        # loss
        error_D_real = criterion(output, labels)
        error_D_real.backward()
        D_x = output.mean().item()

        ## fake
        labels.fill_(0)
        # forward on generator
        fake_now = generator(real_prev)
        fake = torch.hstack((real_prev, fake_now.unsqueeze(1)))
        # forward on discriminator
        output = discriminator(fake.detach())
        # loss
        error_D_fake = criterion(output, labels)
        error_D_fake.backward()
        D_G_z1 = output.mean().item()
        
        # step
        optimizer_D.step()

        #### generator
        labels.fill_(1)
        optimizer_G.zero_grad()
        # calculate discriminator output
        output = discriminator(fake)
        # loss
        error_G = criterion(output, labels)
        error_G.backward()
        D_G_z2 = output.mean().item()
        # step
        optimizer_G.step()
        
        error_D = error_D_real + error_D_fake
        running_D_loss += error_D.item()
        running_G_loss += error_G.item()
        real_up = real_now[:, 0] > real_prev[:, -1, 0]
        fake_up = fake_now[:, 0] > real_prev[:, -1, 0]
        correct += (real_up == fake_up).sum().item()
        cnt += len(data)

    correct /= cnt        
    print(f'epoch {epoch + 1}: g_loss={running_G_loss:.4f}, d_loss={running_D_loss:.4f}, accuracy={correct:.4f}')        
    print(f'D(x): {D_x:.3f}, D(G(z)): {D_G_z1:.3f} {D_G_z2:.3f}')
    print()

100%|██████████| 12/12 [00:00<00:00, 13.59it/s]


epoch 1: g_loss=9.4972, d_loss=17.6637, accuracy=0.4567
D(x): 0.489, D(G(z)): 0.487 0.492



100%|██████████| 12/12 [00:00<00:00, 15.93it/s]


epoch 2: g_loss=8.4515, d_loss=17.0833, accuracy=0.4567
D(x): 0.499, D(G(z)): 0.497 0.496



100%|██████████| 12/12 [00:00<00:00, 15.81it/s]


...

epoch 48: g_loss=51.6376, d_loss=0.4532, accuracy=0.5393
D(x): 0.981, D(G(z)): 0.013 0.013



100%|██████████| 12/12 [00:00<00:00, 15.85it/s]


epoch 49: g_loss=52.9907, d_loss=0.4069, accuracy=0.5380
D(x): 0.983, D(G(z)): 0.013 0.013



100%|██████████| 12/12 [00:00<00:00, 16.02it/s]

epoch 50: g_loss=54.4755, d_loss=0.3567, accuracy=0.5367
D(x): 0.985, D(G(z)): 0.011 0.011



In [ ]:
generator.eval()

Generator(
  (lstm): LSTM(6, 150, batch_first=True)
  (decoder): Sequential(
    (0): Linear(in_features=9000, out_features=2000, bias=True)
    (1): BatchNorm1d(2000, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=2000, out_features=500, bias=True)
    (4): BatchNorm1d(500, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=500, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=6, bias=True)
    (9): ReLU()
  )
)

In [ ]:
correct = 0
real_closing, fake_closing = [], []
for i in range(len(train_ds)):
    data = train_ds[i].to(device)
    data = data.unsqueeze(0)
    real_prev, real_now = data[:, :-1], data[:, -1]
    fake_now = generator(real_prev)
    real_up = real_now[:, 0] > real_prev[:, -1, 0]
    fake_up = fake_now[0] > real_prev[:, -1, 0]
    correct += (real_up == fake_up).item()
    real_closing.append(int(real_up.item()))
    fake_closing.append(int(fake_up.item()))
correct /= len(train_ds)
print(f'Accuracy on train set: {correct}')

Accuracy on train set: 0.5445609436435125


In [ ]:
correct = 0
real_closing, fake_closing = [], []
for i in range(len(test_ds)):
    data = test_ds[i].to(device)
    data = data.unsqueeze(0)
    real_prev, real_now = data[:, :-1], data[:, -1]
    fake_now = generator(real_prev)
    real_up = real_now[:, 0] > real_prev[:, -1, 0]
    fake_up = fake_now[0] > real_prev[:, -1, 0]
    correct += (real_up == fake_up).item()
    real_closing.append(int(real_up.item()))
    fake_closing.append(int(fake_up.item()))
correct /= len(test_ds)
print(f'Accuracy on test set: {correct}')

Accuracy on test set: 0.5037783375314862
